In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from keras.layers import Conv1D , Dropout , Flatten , MaxPooling1D, Dense, Input
from keras.layers.core import Lambda
from keras.models import Model , load_model
import random
import h5py

### Descarga y distribución de datos

In [5]:
# Cargamos los datos
path_local_data = '/home/anell/Desktop/Bovy/AnellExercises/Fits_files'
with h5py.File(f'{path_local_data}/apogeedr14_gaiadr2_with_spectrum_probando_rendimiento_1.h5') as F:  
    parallax = np.array(F['parallax'])
    parallax_error = np.array(F['parallax_err'])
    spectra = np.array(F['spectra'])
    Kmag = np.array(F['corrected_magnitude_K'])
    bp_rp = np.array(F['bp_rp'])
    Gmag = np.array(F['phot_g_mean_mag'])
    teff = np.array(F['NN_teff'])
    apogee_id = np.array(F['APOGEE_ID'])
    snr = np.array(F['SNR'])
    fe_h = np.array(F['Fe/H'])
    path_spectra = np.array(F['Path_spectra'])

/home/anell/anaconda3/envs/python37-astronn/lib/python3.7/site-packages/ipykernel_launcher.py:3: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
parallax.shape , spectra.shape , Kmag.shape

((57696,), (57696, 7514), (57696,))

In [7]:
cut_snr =[]
for i in range(len(snr)):
    if snr[i] > 20:
        cut_snr.append(i)
        
parallax = parallax[cut_snr]
parallax_error = parallax_error[cut_snr]
spectra = spectra[cut_snr]
Kmag = Kmag[cut_snr]
bp_rp = bp_rp[cut_snr]
Gmag = Gmag[cut_snr]
teff = teff[cut_snr]
apogee_id = apogee_id[cut_snr]
snr = snr[cut_snr]
fe_h = fe_h[cut_snr]
path_spectra = path_spectra[cut_snr]

In [8]:
parallax.shape, spectra.shape, Kmag.shape

((57565,), (57565, 7514), (57565,))

In [10]:
# Normalizamos Gmag , el color (G_bp - G_rp) y teff
Gmag_std = np.std(Gmag)
Gmag_mean = np.mean(Gmag)
Gmag_norm = (Gmag - Gmag_mean) / Gmag_std

bp_rp_std = np.std(bp_rp)
bp_rp_mean = np.mean(bp_rp)
bp_rp_norm = (bp_rp - bp_rp_mean) / bp_rp_std

teff_std = np.std(teff)
teff_mean = np.mean(teff)
teff_norm = (teff - teff_mean) / teff_std


#EStablecemos las variables que entrarán a la red y corregimos sus dimensiones
X = np.expand_dims(spectra,axis = 2)
Y = np.expand_dims(parallax,axis=1)
Y_error = np.expand_dims(parallax_error,axis=1)
K_mag = np.expand_dims(Kmag,axis=1)
G_mag = np.expand_dims(Gmag_norm,axis=1)
Bp_Rp = np.expand_dims(bp_rp_norm,axis=1)
Teff = np.expand_dims(teff_norm,axis=1)
Snr = np.expand_dims(snr,axis=1)
FeH = np.expand_dims(fe_h,axis=1)
Teff_without_norm = np.expand_dims(teff,axis=1)

X_offset = np.concatenate((G_mag, Bp_Rp , Teff), axis = 1) 

#Aleatorizamos la muestra
idx = []
for i in range(len(X)):
    idx.append(i)
random.seed(20)
random.shuffle(idx)
    
X = X[idx]                                       # shape: (15644, 7514 , 1)   
Y = Y[idx]                                       # shape: (15644, 1)  
K_mag = K_mag[idx]                               # shape: (15644, 1) 
X_offset = X_offset[idx]                         # shape: (15644, 3)
SNR = Snr[idx]                                   # shape: (15644, 1)
FeH = FeH[idx]                                   # shape: (15644, 1)
Teff_without_norm = Teff_without_norm[idx]       # shape: (15644, 1)

In [11]:
X.shape , Y.shape, parallax.shape

((57565, 7514, 1), (57565, 1), (57565,))

In [13]:
#Definición del modelo
def ApogeeDR14GaiaDR2(dim_t , dim_n): 
    """
    INPUT: 
    dim_t - number of time steps of spectrum 
    dim_n - number of features of spectrum
    """
    
    #SPECTRUM TO LUINOSITY
    dim_1 = 1 # number of corrected magnitude for one example 
    units = 1 #number of final output for one example
    inputs_spectra = Input(shape=(dim_t, dim_n)) 
    inputs_mag = Input(shape=(dim_1,), name="ApparentMagnitude-input")
    
    x_parallax = Conv1D(filters=4, kernel_size=2, activation='relu')(inputs_spectra)
    x_parallax = MaxPooling1D(pool_size=2)(x_parallax)
    x_parallax = Flatten()(x_parallax)
    x_parallax = Dense(164, activation='relu')(x_parallax) #relu
    x_parallax = Dense(164, activation='tanh')(x_parallax) #tanh
    x_parallax = Dense(64, activation='relu')(x_parallax) 
    x_parallax = Dense(64, activation='tanh')(x_parallax)
    x_parallax = Dense(32, activation='relu')(x_parallax)
    x_parallax = Dense(units, activation='softplus')(x_parallax)
    
    outputs_parallax = Lambda(lambda function: tf.math.multiply(function[0], tf.math.pow(10., 
                              tf.math.multiply(-0.2, function[1]))),
                              name='luminosity-to-parallax')([x_parallax, inputs_mag])
   
    #OFFSET CORRECTION : (optimization)
    inputs_offset = Input(shape=(3,), name="Offset-input")
    x_offset = Dense(64, activation='relu')(inputs_offset)
    x_offset = Dense(32, activation='relu')(x_offset) 
    x_offset = Dense(units, activation='tanh')(x_offset) 
    
    outputs_parallax_with_offset = Lambda(lambda function: tf.math.add(function[0], function[1]),
                                          name="Sum-parallax-offset")([outputs_parallax, x_offset]) 
    
    #Model setup
    model =  Model(inputs = [inputs_spectra,inputs_mag, inputs_offset],outputs = [outputs_parallax_with_offset])
    
    return model 

In [15]:
#Formación del set de entrenamiento
good_idx_snr = []
bad_idx_snr = []
for i in range(len(SNR)):
    if SNR[i] >= 200:           
        good_idx_snr.append(i)
    else:
        bad_idx_snr.append(i)

#SET ENTRENAMIENTO
X_train_snr = np.concatenate((X[good_idx_snr][:4000],X[bad_idx_snr][:2000]),axis = 0)
Y_train_snr = np.concatenate((Y[good_idx_snr][:4000],Y[bad_idx_snr][:2000]),axis = 0)
K_mag_train_snr = np.concatenate((K_mag[good_idx_snr][:4000],K_mag[bad_idx_snr][:2000]),axis = 0)
X_offset_train_snr = np.concatenate((X_offset[good_idx_snr][:4000],X_offset[bad_idx_snr][:2000]),axis = 0)

idx_random = []
for i in range(len(X_train_snr)):
    idx_random.append(i)
random.seed(1000)
random.shuffle(idx_random)

X_train_snr = X_train_snr[idx_random]
Y_train_snr = Y_train_snr[idx_random]
K_mag_train_snr = K_mag_train_snr[idx_random]
X_offset_train_snr = X_offset_train_snr[idx_random]

#SET DE VALIDACIÓN
#X_val_snr = np.concatenate((X[good_idx_snr][4000:5500],X[bad_idx_snr][2000:2500]),axis = 0)
#Y_val_snr = np.concatenate((Y[good_idx_snr][4000:5500],Y[bad_idx_snr][2000:2500]),axis = 0)
#K_mag_val_snr = np.concatenate((K_mag[good_idx_snr][4000:5500],K_mag[bad_idx_snr][2000:2500]),axis = 0)
#X_offset_val_snr = np.concatenate((X_offset[good_idx_snr][4000:5500],X_offset[bad_idx_snr][2000:2500]),axis = 0)

#idx_random_val = []
#for i in range(len(X_val_snr)):
#    idx_random_val.append(i)
#random.seed(5000)
#random.shuffle(idx_random_val)

#X_val_snr = X_val_snr[idx_random_val]
#Y_val_snr = Y_val_snr[idx_random_val]
#K_mag_val_snr = K_mag_val_snr[idx_random_val]
#X_offset_val_snr = X_offset_val_snr[idx_random_val]

In [16]:
n_timesteps, n_features = X_train_snr.shape[1], X_train_snr.shape[2]

Global_model = ApogeeDR14GaiaDR2(n_timesteps , n_features)

Global_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 7514, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 7513, 4)      12          input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 3756, 4)      0           conv1d[0][0]                     
__________________________________________________________________________________________________
flatten (Flatten)               (None, 15024)        0           max_pooling1d[0][0]              
_______________________________________________________________________________________

In [17]:
#Paré el entrenamiento para que no se sobreajustara, ya que aún no tenía la visión del comportamiento del set de validación
Global_model.compile(optimizer='adam', loss='mse', metrics=['mse'])
Global_model.fit([X_train_snr, K_mag_train_snr , X_offset_train_snr], Y_train_snr, epochs=300, batch_size=128, verbose=1)


Epoch 1/300
47/47 [==============================] - 2s 50ms/step - loss: 14.1851 - mse: 14.1851
Epoch 2/300
47/47 [==============================] - 2s 50ms/step - loss: 12.7362 - mse: 12.7362
Epoch 3/300
47/47 [==============================] - 2s 49ms/step - loss: 12.2781 - mse: 12.2781
Epoch 4/300
47/47 [==============================] - 2s 50ms/step - loss: 11.3458 - mse: 11.3458
Epoch 5/300
47/47 [==============================] - 2s 50ms/step - loss: 10.3248 - mse: 10.3248
Epoch 6/300
47/47 [==============================] - 2s 50ms/step - loss: 9.2651 - mse: 9.2651
Epoch 7/300
47/47 [==============================] - 2s 50ms/step - loss: 8.2702 - mse: 8.2702
Epoch 8/300
47/47 [==============================] - 2s 49ms/step - loss: 7.4124 - mse: 7.4124
Epoch 9/300
47/47 [==============================] - 2s 50ms/step - loss: 6.6713 - mse: 6.6713
Epoch 10/300
47/47 [==============================] - 2s 51ms/step - loss: 5.9303 - mse: 5.9303
Epoch 11/300
47/47 [===================

KeyboardInterrupt: 

In [18]:
Global_model.save('probando_rendimiento_1_SNR(4000-2000).h5')

In [ ]:
#Global_model.compile(optimizer='adam', loss='mse', metrics=['mse'])

#path_local = "/home/anell/Desktop/AnellExercises/Models/"

#callbacks = [ModelCheckpoint(f'{path_local}/model_probando_rendimiento_1_SNR(4000-2000)_2.h5', verbose=1, save_best_only=True)]

#Global_model.fit([X_train_snr,K_mag_train_snr,X_offset_train_snr], Y_train_snr, batch_size=128, #shuffle='batch', 
#                 epochs=300, callbacks=callbacks, 
#                 validation_data=([X_val_snr,K_mag_val_snr,X_offset_val_snr], Y_val_snr))